In [1]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

In [3]:
# Create my_spark or get it if already created
spark = SparkSession.builder.getOrCreate()

In [4]:
#reading multiple csv in a directory
df = spark.read.csv(['C:/Users/IP 320/Desktop/AUEB Assisgments/Dataset'],\
                    header=True, \
                    inferSchema = True)

There are a lot of information in the dataset. For the assignment purpose, we will extract only the columns that might be useful to help us understand and answer from WHO, to WHO, and WHEN.

Before we start on the aggregation, we need to understand our dataset.

We will check the following and do the necessary transformation accordingly.

- 1) Verify col and rows
- 2) Datatype
- 3) WhiteSpaces
- 4) Remove any duplicate values

In [5]:
# Let's print the schema to understand which variables are in the df and their types
df.printSchema()

root
 |-- NAMELAST: string (nullable = true)
 |-- NAMEFIRST: string (nullable = true)
 |-- NAMEMID: string (nullable = true)
 |-- UIN: string (nullable = true)
 |-- BDGNBR: string (nullable = true)
 |-- ACCESS_TYPE: string (nullable = true)
 |-- TOA: string (nullable = true)
 |-- POA: string (nullable = true)
 |-- TOD: string (nullable = true)
 |-- POD: string (nullable = true)
 |-- APPT_MADE_DATE: string (nullable = true)
 |-- APPT_START_DATE: string (nullable = true)
 |-- APPT_END_DATE: string (nullable = true)
 |-- APPT_CANCEL_DATE: string (nullable = true)
 |-- Total_People: string (nullable = true)
 |-- LAST_UPDATEDBY: string (nullable = true)
 |-- POST: string (nullable = true)
 |-- LastEntryDate: string (nullable = true)
 |-- TERMINAL_SUFFIX: string (nullable = true)
 |-- visitee_namelast: string (nullable = true)
 |-- visitee_namefirst: string (nullable = true)
 |-- MEETING_LOC: string (nullable = true)
 |-- MEETING_ROOM: string (nullable = true)
 |-- CALLER_NAME_LAST: string (

In [6]:
total_records = df.count()
print("The number of records are", total_records)

The number of records are 3727296


A few points to highlight.
    - Visitor made an appointment and was given a unique appointment number(UIN)
    - Appointment can be canceled
    - Time of vistors arrival who actually turned up at whitehouse is recorded by the TOA column.

Staff at the whitehouse cannot predict if the applicants will turn up or not, thus we can see that there are many missing values in the TOA colum.

It may be useful for staff to plan the logistics requirement such as available rooms, security, and human traffic flow according to scheduled appointment start date. In this report, we will APPT_START DATE given that APPT_CANCEL_DAT is not null to analysis and answer the questions given.

In [7]:
df.select('NAMEFIRST','TOA','POA','Total_People','APPT_START_DATE','APPT_CANCEL_DATE','UIN').filter(df.Total_People.isNotNull()).show(30)

+---------+--------------+-----+------------+---------------+----------------+------+
|NAMEFIRST|           TOA|  POA|Total_People|APPT_START_DATE|APPT_CANCEL_DATE|   UIN|
+---------+--------------+-----+------------+---------------+----------------+------+
|   Donett| 2/26/12 18:50|B0402|           8|  2/26/12 19:00|            null|U83356|
|    James| 2/26/12 18:50|B0402|           8|  2/26/12 19:00|            null|U83356|
|Katherine|          null| null|           1|  5/18/12 11:00|            null|U04962|
|Katherine| 5/24/12 14:49| K101|           1|  5/24/12 15:00|            null|U08899|
|     Kirk|          null| null|          43|  9/18/12 11:30|            null|U39090|
|    Dylan|          null| null|         268|  10/5/12 12:30|            null|U41863|
|  Jeffrey|          null| null|         268|  10/5/12 12:30|            null|U41863|
|    Robyn|          null| null|         268|  10/5/12 12:30|            null|U41863|
| Kimberly|          null| null|         254| 12/15/12

In [24]:
#selecting the relevant ifnormation that we require for the analysis
cols_select =['NAMELAST',
             'NAMEFIRST',
             'APPT_START_DATE',
             'APPT_CANCEL_DATE',
             'Total_People',
             'visitee_namelast',
             'visitee_namefirst',
             'UIN']
data = df.select(cols_select)

In [25]:
data.show(20)

+--------+---------+---------------+----------------+------------+----------------+-----------------+------+
|NAMELAST|NAMEFIRST|APPT_START_DATE|APPT_CANCEL_DATE|Total_People|visitee_namelast|visitee_namefirst|   UIN|
+--------+---------+---------------+----------------+------------+----------------+-----------------+------+
|    Aaby|   Donett|  2/26/12 19:00|            null|           8|            Aaby|             Kate|U83356|
|    Aaby|    James|  2/26/12 19:00|            null|           8|            Aaby|             Kate|U83356|
|    Aaby|Katherine|  5/18/12 11:00|            null|           1|            Ross|             Adam|U04962|
|    Aaby|Katherine|  5/24/12 15:00|            null|           1|            Ross|             Adam|U08899|
| Aadalen|     Kirk|  9/18/12 11:30|            null|          43|            null|            POTUS|U39090|
|  AAFEDT|  ANTHONY|    6/8/12 8:00|            null|        null|            null|             null|U13475|
|  AAFEDT|     CODY

Handling date timestamp, Filling NAs and Casting Datatypes

We will handle them one by one.

First we start off by converting string type data which should be timestamp.

In [26]:
#Gives us the idea of datatype and perform necessary transformation.
data.dtypes

[('NAMELAST', 'string'),
 ('NAMEFIRST', 'string'),
 ('APPT_START_DATE', 'string'),
 ('APPT_CANCEL_DATE', 'string'),
 ('Total_People', 'string'),
 ('visitee_namelast', 'string'),
 ('visitee_namefirst', 'string'),
 ('UIN', 'string')]

In [27]:
from pyspark.sql.functions import dayofmonth,from_unixtime,month, unix_timestamp, year,date_format, to_timestamp, col

#the datatype of the column 'APPT_START_DATE' is string
#I do the conversion and created column datetime2 which is time stamp

data = data.withColumn('datetime2', to_timestamp(col('APPT_START_DATE'), "MM/dd/yy").cast('timestamp'))

In [28]:
#extract month, year, day information
#create new column for each row of day, month, year

data = data.withColumn("Appt_month", month(data.datetime2)).\
            withColumn("Appt_year", year(data.datetime2)).\
            withColumn("Appt_day", dayofmonth(data.datetime2))

In [29]:
#extract day of week
#create new columns of Day of week

data = data.withColumn("DayOfWeek", date_format("datetime2","EEEE")).\
      withColumn("DayOfWeek_number", date_format("datetime2","u")) # and numerics

Next, we tackle the string values that should be numeric values by casting them to Integer type.

In [30]:
from pyspark.sql.types import IntegerType
data = data.withColumn("Total_People", data["Total_People"].cast(IntegerType())).\
            withColumn("DayofWeek_number", data["DayofWeek_number"].cast(IntegerType()))

In [31]:
from pyspark.sql.functions import trim

data = data.withColumn('visitee_namelast', trim(data.visitee_namelast))
data = data.withColumn('visitee_namefirst', trim(data.visitee_namefirst))

As mentioned above, we want to analysis those appointments that is not cancelled, so we filter out the Null values in APPT_CANCEL_DATE. 

We also drop the columns that are no longer useful as we have created new ones with the correct data type.

In [32]:
# Filter by taking the rows that are null
data2 = data.filter(data.APPT_CANCEL_DATE.isNull())

In [33]:
data = data2.drop('APPT_START_DATE','APPT_CANCEL_DATE','datetime2')

In [35]:
#data_agg = data.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in data.columns])
#data_agg.show()

+--------+---------+----------------+-----------------+-----+----+---+---------+-------------+-----------------+
|NAMELAST|NAMEFIRST|visitee_namelast|visitee_namefirst|month|year|day|DayOfWeek|Total_People2|DayofWeek_number2|
+--------+---------+----------------+-----------------+-----+----+---+---------+-------------+-----------------+
|       0|        0|               0|                0|    0|   0|  0|        0|            0|                0|
+--------+---------+----------------+-----------------+-----+----+---+---------+-------------+-----------------+



After all the steps above, we are finally ready to move on to our questions!

The first question is asking us to find the top 20 visitor. We did a groupby First and Last name and order the count of each names in descending order. This information disregard the period they visited the Whitehouse but we can be sure they are frequent vistors.

In [34]:
#1.1 Who are the top 20 visitors?
by_visitor = data.select(data.NAMELAST, data.NAMEFIRST).groupBy("NAMELAST","NAMEFIRST")

In [35]:
#.count() in pyspark is a lazy operation which literally counts every row so it took a long time.
by_visitor.count().orderBy("count", ascending=False).show(20)

+------------+---------+-----+
|    NAMELAST|NAMEFIRST|count|
+------------+---------+-----+
|        Hash|  Michael|  726|
|    Tavenner|  Marilyn|  519|
|        Hoff|    James|  482|
|BrooksLaSure| Chiquita|  410|
|     Levitis|    Jason|  384|
|       Borzi|  Phyllis|  368|
|    Fontenot|   Yvette|  350|
|      Khalid|   Aryana|  341|
|        Mann|  Cynthia|  323|
|      Werner|   Sharon|  310|
|        Choe|  Kenneth|  305|
|       Smith|  Michael|  304|
|       Jones|   Daniel|  303|
|      Turner|      Amy|  302|
|    Morrison|    Helen|  295|
|  Livingston|Catherine|  295|
|     Kronick|  Richard|  292|
|       Lewis|     Caya|  292|
|     Maguire|   Daniel|  290|
|     Schultz|  William|  283|
+------------+---------+-----+
only showing top 20 rows



During the period when Obama is the president, he is affectionally referred to as "POTUS" which stands for President of the United States and first lady Michelle Obama as "FLOTUS" which stands for First Lady of the United States.

From our results, we can see that the majority of the vistors visted the vistors office. During these period, Obama welcomed visitors to the whitehouse and frequently organizes group tours, parties for Halloween and Christmas and many other ocassions. We are not surprise to see as well they are also among the top visitee. 

Kyle Lierman serves as a Senior Associate Director of Public Engagement and Senior Policy Advisor in the White House Office of Public Engagement. Perhaps he is busy with the media, and reporters I would assume. 

In [47]:
from pyspark.sql.functions import upper

data = data.withColumn('visitee_namefirst', upper(col('visitee_namefirst')))
data = data.withColumn('visitee_namelast', upper(col('visitee_namelast')))

In [48]:
#1.2 Who are the top 20 visitees?
#Filter and GroupBy visitee

by_visitee2 = data.select('visitee_namefirst','visitee_namelast').\
            filter(data.visitee_namefirst.isNotNull()).\
            groupBy('visitee_namefirst','visitee_namelast').\
            count().\
            orderBy("count", ascending=False).\
            show(20)

+-----------------+----------------+-------+
|visitee_namefirst|visitee_namelast|  count|
+-----------------+----------------+-------+
|         VISITORS|          OFFICE|2271048|
|            POTUS|            null|  98674|
|     POTUS/FLOTUS|            null|  24155|
|             KYLE|         LIERMAN|  15590|
|           JEANNE|         LAMBREW|  14667|
|            POTUS|               /|  10876|
|           OFFICE|          HETZEL|  10672|
|             PAUL|        MONTEIRO|   9549|
|           FLOTUS|            null|   9319|
|              MAX|         DOEBLER|   9238|
|           GAUTAM|        RAGHAVAN|   7468|
|             GREG|          NELSON|   7203|
|         VICTORIA|      MCCULLOUGH|   7127|
|          HEATHER|          FOSTER|   7062|
|           LAUREN|           KELLY|   6427|
|            NANCY|           HOGAN|   6422|
|            KAREN|      RICHARDSON|   5666|
|              JON|          CARSON|   4834|
|            JULIE|       RODRIGUEZ|   4736|
|         

For this third question, we would like to understand who gets the most appointments from who. 
We follow the steps to select the names of visitee and visitors and do a groupby and count the number of times they have scheduled an appointment. Since we are not particular on the dates, we will omit this information. 

Again, we order the count in descending order to see the top pairs. In the result, we see that Jeanne Lambrew is very busy with many people and Joanne Hoff gets most of her appointments from James Hoff.

I decide to dig a bit further and found some information why James Hoff is visiting Joanne Hoff. We may look at it at the link below.

https://www.reddit.com/r/RBI/comments/44thud/who_is_james_c_hoff_he_has_had_the_third_most/

In [49]:
#1.3 Who are the top 20 visitor-visitee pairs?
by_pairs = data.select("NAMELAST", "NAMEFIRST", "visitee_namelast", "visitee_namefirst").groupBy("NAMELAST","NAMEFIRST","visitee_namelast","visitee_namefirst")

In [50]:
by_pairs.count().orderBy("count", ascending=False).show(20)

+------------+---------+----------------+-----------------+-----+
|    NAMELAST|NAMEFIRST|visitee_namelast|visitee_namefirst|count|
+------------+---------+----------------+-----------------+-----+
|        Hash|  Michael|         LAMBREW|           JEANNE|  532|
|        Hoff|    James|            HOFF|           JOANNE|  387|
|    Tavenner|  Marilyn|         LAMBREW|           JEANNE|  382|
|BrooksLaSure| Chiquita|         LAMBREW|           JEANNE|  334|
|    Fontenot|   Yvette|         LAMBREW|           JEANNE|  293|
|     Levitis|    Jason|         LAMBREW|           JEANNE|  291|
|      Khalid|   Aryana|         LAMBREW|           JEANNE|  289|
|        Mann|  Cynthia|         LAMBREW|           JEANNE|  276|
|     Kronick|  Richard|         LAMBREW|           JEANNE|  253|
|        Choe|  Kenneth|         LAMBREW|           JEANNE|  249|
|  Livingston|Catherine|         LAMBREW|           JEANNE|  241|
|       Borzi|  Phyllis|         LAMBREW|           JEANNE|  241|
|     Magu

Now, we are answering the question for WHEN and we are using the dates. We will try two different ways trying to see each results.

In the first one, we count the number of appointments considering that it will be important for the staff to how many groups. 

In the second one, we sum up the number of people visiting that day. Of course again we dont know if they will cancel the appointment. This will help to see how crowded the Whitehouse will be.

In [59]:
import pyspark.sql.functions as F

In [68]:
#1.4 What were the top 20 most busy days?

busiest_day = data.select("Appt_day","Appt_month","Appt_year","Total_People").\
            filter(data.Appt_day.isNotNull()).\
            filter(data.Appt_month.isNotNull()).\
            groupBy("Appt_day","Appt_month","Appt_year").\
            count().\
            orderBy('count', ascending= False).show(20)

+--------+----------+---------+-----+
|Appt_day|Appt_month|Appt_year|count|
+--------+----------+---------+-----+
|      14|         3|     2012| 9135|
|       9|        12|     2009| 8966|
|      18|        12|     2010| 8876|
|      19|         5|     2010| 8740|
|      11|        12|     2009| 8689|
|      18|         8|     2011| 8606|
|      11|        12|     2010| 8387|
|      21|        12|     2010| 8227|
|      22|        12|     2010| 8215|
|       7|        12|     2013| 8092|
|      18|        12|     2009| 8079|
|      19|        12|     2012| 7833|
|      18|         6|     2010| 7815|
|       4|        12|     2009| 7783|
|       8|        12|     2012| 7477|
|      14|        12|     2013| 7464|
|      15|        12|     2012| 7426|
|      20|        12|     2010| 7420|
|      21|        12|     2012| 7412|
|      23|        12|     2009| 7355|
+--------+----------+---------+-----+
only showing top 20 rows



In [64]:
busiest_day2 = data.select("Appt_day","Appt_month","Appt_year","Total_People").\
            filter(data.Appt_day.isNotNull()).\
            filter(data.Appt_month.isNotNull()).\
            groupBy("Appt_day","Appt_month","Appt_year")

In [65]:
busiest_day2.agg(F.sum(data.Total_People)).orderBy("sum(Total_People)", ascending = False).show()

+--------+----------+---------+-----------------+
|Appt_day|Appt_month|Appt_year|sum(Total_People)|
+--------+----------+---------+-----------------+
|      14|         3|     2012|         33946485|
|      19|         5|     2010|         21507390|
|       8|         9|     2011|         17779960|
|      29|         6|     2010|         17389336|
|       9|        12|     2009|         16717218|
|       7|         6|     2011|         16631811|
|      13|        10|     2011|         16509741|
|       7|         4|     2013|         16251305|
|       4|         7|     2012|         14286691|
|      22|         1|     2013|         14031755|
|       7|        12|     2009|         13735919|
|       4|         7|     2010|         13090176|
|      11|        12|     2009|         12688467|
|       4|         7|     2011|          9490141|
|      10|        12|     2009|          8158890|
|       4|        12|     2009|          7352342|
|      31|        10|     2010|          7202542|


We can see that 2009 December is the most busiest month in the year. It is the first inauguration of Barack Obama as the 44th President of the United States took place on Tuesday, January 20, 2009. And in December, the first family organizes their first Christmas events, and many other events that year. 

In 2012, there's an election and Obama participated in it and won. It did not happen in March 2012 but that month is also second busiest month.

https://obamawhitehouse.archives.gov/realitycheck/briefing-room/Statements-and-Releases/2009/09?page=8

In [70]:
#1.5 What were the top 20 most busy months-years?
busiest_month = data.select("Appt_month","Appt_year","Total_People").groupBy("Appt_month","Appt_year")
busiest_month.agg(F.sum(data.Total_People)).orderBy("sum(Total_People)", ascending = False).show()

+----------+---------+-----------------+
|Appt_month|Appt_year|sum(Total_People)|
+----------+---------+-----------------+
|        12|     2009|         85846825|
|         3|     2012|         51540240|
|         6|     2010|         44182242|
|         6|     2011|         40340230|
|      null|     null|         40272335|
|         5|     2010|         40084890|
|        10|     2011|         35322342|
|        12|     2010|         34723022|
|         7|     2010|         34715110|
|         7|     2011|         31705584|
|         9|     2011|         31678363|
|         7|     2012|         30639110|
|        12|     2012|         28373883|
|        12|     2011|         26899485|
|         3|     2010|         26632273|
|        10|     2010|         25563169|
|        12|     2013|         22655331|
|        10|     2009|         21165038|
|         5|     2011|         20966141|
|         3|     2011|         20745076|
+----------+---------+-----------------+
only showing top

Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday. Which day is the favourite day to visit the whitehouse?

Firstly, we checked the spelling if the system is correct. They are okie. 

Next we took the total of the number of people visited based on the DayofWeek column. Not surprising they are not a fan of Monday events. Of course no one like weekend work and appointments so it's normal that weekends does not have so much counts.

In [72]:
#1.6 What was the order of popularity of days of week for visits?
weekDaysCount  = data.filter(data.DayOfWeek.isNotNull()).\
                groupBy(["DayOfWeek", "DayOfWeek_number"]).\
                count()

weekDaysCount.show()

+---------+----------------+------+
|DayOfWeek|DayOfWeek_number| count|
+---------+----------------+------+
| Saturday|               6|685829|
|   Sunday|               7|112339|
|   Friday|               5|726778|
|   Monday|               1|232728|
|Wednesday|               3|616971|
|  Tuesday|               2|637941|
| Thursday|               4|623846|
+---------+----------------+------+



In [74]:
popular_DayofWeek = data.select("DayOfWeek","DayOfWeek_number","Total_People").filter(data.DayOfWeek.isNotNull()).groupBy("DayOfWeek","DayOfWeek_number")
popular_DayofWeek.agg(F.sum(data.Total_People)).orderBy("sum(Total_People)", ascending = False).show()

+---------+----------------+-----------------+
|DayOfWeek|DayOfWeek_number|sum(Total_People)|
+---------+----------------+-----------------+
|Wednesday|               3|        205640558|
|  Tuesday|               2|        180589415|
| Saturday|               6|        179292977|
|   Friday|               5|        178059876|
| Thursday|               4|        161285552|
|   Monday|               1|         61812155|
|   Sunday|               7|         41035499|
+---------+----------------+-----------------+



And now we are at our last question on Whitehouse before moving on the machine learning portion. We want to understand which month is the most popular month to visit. Regardless of year because this time, we want to understand if there are any seasons or period which has the highest visitors.

December is top on the list for highest appointments made and most number of visitors to the whitehouse. December is the christmas seasons and so is the whitehouse and first lady welcoming their guest and visitors.

In [77]:
#1.7 What was the order of popularity of months for visits?
monthCount  = data.filter(data.Appt_month.isNotNull()).groupBy(["Appt_month"]).count()
monthCount.show()

+----------+------+
|Appt_month| count|
+----------+------+
|        12|556349|
|         1|218811|
|         6|320055|
|         3|403654|
|         5|280021|
|         9|239251|
|         4|277436|
|         8|261852|
|         7|266468|
|        10|329885|
|        11|269890|
|         2|212760|
+----------+------+



In [78]:
popular_MonthofYear = data.select("Appt_month","Total_People").filter(data.Appt_month.isNotNull()).groupBy("Appt_month")
popular_MonthofYear.agg(F.sum(data.Total_People)).orderBy("sum(Total_People)", ascending = False).show()

+----------+-----------------+
|Appt_month|sum(Total_People)|
+----------+-----------------+
|        12|        198498546|
|         3|        121285047|
|        10|         99627265|
|         7|         97128915|
|         6|         86522222|
|         5|         77416391|
|         4|         73297811|
|         8|         59129261|
|         9|         58996443|
|        11|         53462012|
|         1|         46034491|
|         2|         36317628|
+----------+-----------------+

